In [2]:
import cv2
import mediapipe as mp
import os
import numpy as np
import math as m

2023-06-08 21:24:15.007852: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
font = cv2.FONT_HERSHEY_SIMPLEX

In [4]:
# 경로 지정
root_dir = 'crops2'

# 불러올 파일의 확장자 지정(여기서는 이미지 파일만 로드)
img_path_list = []
possible_img_extension = ['.jpg', '.jpeg','.png']

for (root, dirs, files) in os.walk(root_dir):
    if len(files) > 0:
        for file_name in files:
            if os.path.splitext(file_name)[1] in possible_img_extension:
                img_path = root + '/' + file_name
                
                # 경로에서 \를 모두 /로 바꿔줌
                img_path = img_path.replace('\\', '/') # \는 \\로 나타내야함         
                img_path_list.append(img_path)


# tree 관절 각도를 저장할 리스트 선언
tree_right_angle_list = []
tree_right_knee_angle_list = []
tree_left_angle_list = []
tree_left_knee_angle_list = []

# cobra 관절 각도를 저장할 리스트 선언
cobra_hip_angle_list = []
cobra_neck_angle_list = []

In [5]:
# 각도 계산 함수
def calculate_angle(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c) 
    
    # 역탄젠트를 이용한 라디언값 도출
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    # 라디언값 -> 도단위
    angle = np.abs(radians*180.0/np.pi)
    
    # 모든 동작에서 180도를 넘어가진 않으므로 180도 이하의 값만 도출
    if angle >180.0:
        angle = 360-angle
        
    return angle

In [6]:
with mp_pose.Pose(
        static_image_mode=True,
        model_complexity=2,
        enable_segmentation=True,
        min_detection_confidence=0.5) as pose:
    
# 파일 경로에서 이미지 가져오기
    for idx, file in enumerate(img_path_list):
            
        image = cv2.imread(file)
        image_height, image_width, _ = image.shape
        h, w = image.shape[:2]
        
        
        # 처리 전 opencv를 통해 불러온 이미지를 BGR 순서에서 RGB 순서로 변환
        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        if not results.pose_landmarks:
            continue
        
        landmarks = results.pose_landmarks.landmark
        
        # mediapipe를 통해 관절 좌표 로드
        left_shoulder = [int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x*w),
                         int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y*h)]
        right_hip = [int(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x*w),
                    int(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y*h)]
        left_hip = [int(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x*w),
                    int(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y*h)]
        right_knee = [int(landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x*w),
                     int(landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y*h)]
        left_knee = [int(landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x*w),
                     int(landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y*h)]
        left_ankle = [int(landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x*w),
                      int(landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y*h)]
        right_heel = [int(landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].x*w),
                     int(landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y*h)]
        left_heel = [int(landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x*w),
                     int(landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y*h)]
        left_foot_index = [int(landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x*w),
                           int(landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y*h)]
        right_shoulder = [int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x*w),
                          int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y*h)]
        right_elbow = [int(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x*w),
                          int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y*h)]
        left_elbow = [int(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x*w),
                          int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y*h)]
        right_wrist = [int(landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x*w),
                      int(landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y*w)]
        left_wrist = [int(landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x*w),
                      int(landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y*w)]
        right_eye = [int(landmarks[mp_pose.PoseLandmark.RIGHT_EYE.value].x*w),
                      int(landmarks[mp_pose.PoseLandmark.RIGHT_EYE.value].y*w)]
        nose = [int(landmarks[mp_pose.PoseLandmark.NOSE.value].x*w),
                      int(landmarks[mp_pose.PoseLandmark.NOSE.value].y*w)]
        
        
        # 정확한 자세인지 확인하기 위해 양쪽 무릅, 발목 각도 계산
        tree_right_angle = int((calculate_angle(right_shoulder, right_hip, right_knee)))
        tree_right_knee_angle = int((calculate_angle(right_hip, right_knee,right_heel)))
        tree_left_knee_angle = int((calculate_angle(left_hip, left_knee,left_heel)))
        tree_left_angle = int((calculate_angle(left_shoulder, left_hip, left_knee)))
        
        
        # cobra 데이터 각도 계산
        cobra_hip_angle1 = int((calculate_angle(right_hip, right_shoulder, right_eye)))
        cobra_neck_angle1 = int((calculate_angle(right_elbow, right_shoulder, right_eye)))
        
        # tree 데이터 각도를 각 리스트에 append
        tree_right_angle_list.append(tree_right_angle)
        tree_right_knee_angle_list.append(tree_right_knee_angle)
        tree_left_knee_angle_list.append(tree_left_knee_angle)
        tree_left_angle_list.append(tree_left_angle) 
        
        # cobra 데이터 각도를 각 리스트에 append
        cobra_hip_angle_list.append(cobra_hip_angle1)
        cobra_neck_angle_list.append(cobra_neck_angle1)
        
        
        
# tree 각도의 평균값
tree_right_angle_avg=0
tree_right_knee_angle_avg=0
tree_left_knee_angle_avg=0
tree_left_angle_avg=0

# cobra 데이터 각도 평균값
cobra_hip_angle_avg = 0
cobra_neck_angle_avg = 0


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
libpng warning: iCCP: known incorrect sRGB profile


In [7]:
# 리스트의 평균을 구하는 함수 생성
def list_average(list):
    avg=0
    sum=0
    for i in list:
        sum+=i
        avg=int(sum/len(list))
    return avg

In [8]:
#tree 각도 평균값
tree_right_angle_avg=list_average(tree_right_angle_list)
tree_right_knee_angle_avg=list_average(tree_right_knee_angle_list)
tree_left_knee_angle_avg = list_average(tree_left_knee_angle_list)
tree_left_angle_avg=list_average(tree_left_angle_list)

# cobra 각도 평균값
cobra_hip_angle_avg = list_average(cobra_hip_angle_list)
cobra_neck_angle_avg = list_average(cobra_neck_angle_list)

In [9]:
tree_right_angle_avg

127

In [10]:
tree_right_knee_angle_avg

164

In [11]:
tree_left_knee_angle_avg

164

In [12]:
tree_left_angle_avg

128

In [13]:
cobra_hip_angle_avg

119

In [14]:
cobra_neck_angle_avg 

107

In [15]:
# 시각화를 위한 색상 변수 선언
white = (255, 255, 255)
yellow = (50,180,225)
red = (50, 50, 255)
green = (127, 255, 0)
light_green = (125, 233, 100)

In [34]:
#cv를 통해 동영상 불러오기
cap = cv2.VideoCapture('jun_cobra.mp4')

# bae_tree
# msik_tree
# suji_tree

# h_cobra
# oh_cobra
# jun_cobra 

# 초기화 변수 설정
yoga_pose = "nothing"
state = "normal" 
aligned_state = 'all_body'

# 사람 객체가 측면인지 확인하는 함수
def direction(x1, y1, x2, y2):
    dist = m.sqrt((x1-x2)**2+(y1-y2)**2)
    return dist

# 퍼펙트할때 마다 1씩 커지는 count와 보조하는 count_condition 변수, 트리와 코브라 동작에서 측정 불가 side를 구분하는 condition 변수
count = 0
count_condition = True
condition = 'neutral'

In [36]:

if __name__ == "__main__":

    
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        
        while cap.isOpened():
            
            ret, frame = cap.read()

            if not ret:
                break

            # opencv를 통해 불러온 이미지를 BGR 순서에서 RGB 순서로 변환
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            h, w = image.shape[:2]

            # 탐지하기
            results = pose.process(image)

            # cv2를 통해 출력하기 위해 RGB 순서의 이미지를 다시 BGR로 변환
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)        
            cobra_hip_angle = 0
            
            
            # 감지 시작
            try:
                landmarks = results.pose_landmarks.landmark

                # mediapipe를 통한 관절 좌표
           
                left_shoulder = [int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x*w),
                         int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y*h)]
                right_hip = [int(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x*w),
                    int(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y*h)]
                left_hip = [int(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x*w),
                    int(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y*h)]
                right_knee = [int(landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x*w),
                     int(landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y*h)]
                left_knee = [int(landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x*w),
                     int(landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y*h)]
                right_elbow = [int(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x*w),
                      int(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y*h)]
                left_elbow = [int(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x*w),
                      int(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y*h)]
                right_heel = [int(landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].x*w),
                     int(landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y*h)]
                left_heel = [int(landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x*w),
                     int(landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y*h)]
                left_foot_index = [int(landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x*w),
                           int(landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y*h)]
                right_foot_index = [int(landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x*w),
                           int(landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y*h)]
                right_shoulder = [int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x*w),
                          int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y*h)]
                right_wrist = [int(landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x*w),
                      int(landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y*w)]
                left_wrist = [int(landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x*w),
                      int(landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y*w)]
                right_eye = [int(landmarks[mp_pose.PoseLandmark.RIGHT_EYE.value].x*w),
                      int(landmarks[mp_pose.PoseLandmark.RIGHT_EYE.value].y*w)]
                nose = [int(landmarks[mp_pose.PoseLandmark.NOSE.value].x*w),
                      int(landmarks[mp_pose.PoseLandmark.NOSE.value].y*w)]
        
                # tree 각도 계산
                tree_right_angle = int((calculate_angle(right_shoulder, right_hip, right_knee)))
                tree_right_knee_angle = int((calculate_angle(right_hip, right_knee,right_heel)))
                tree_left_knee_angle = int((calculate_angle(left_hip, left_knee,left_heel)))
                tree_left_angle = int((calculate_angle(left_shoulder, left_hip, left_knee)))
        
                # tree 각도 리스트에 담기
                tree_right_angle_list.append(tree_right_angle)
                tree_right_knee_angle_list.append(tree_right_knee_angle)
                tree_left_knee_angle_list.append(tree_left_knee_angle)
                tree_left_angle_list.append(tree_left_angle)
                
                
                # cobra 각도 계산
                cobra_hip_angle = int((calculate_angle(right_hip, right_shoulder, right_eye)))

                # cobra 각도 리스트에 담기
                cobra_hip_angle_list.append(cobra_hip_angle)

                
            
                #신체 측면이 촬영되고 있는지 확인하기 위해 양쪽 어깨의 거리를 계산
                body_check = direction(left_shoulder[1], left_shoulder[0], right_shoulder[1] , right_shoulder[0])
                
                tree_check = right_hip[0] - right_eye[0]
                
                cobra_check = right_heel[1] - right_hip[1]
                cobra_check1 = right_shoulder[0] - right_heel[0]


                # 양쪽 어깨의 거리가 40 이하이면 side_body 상태 및 코브라 동작 감지
                if body_check < 110:
                    body_state = 'side_body'
                    
                    if (cobra_hip_angle >= cobra_hip_angle_avg-20) and (nose[1] <= left_wrist[1]) and (cobra_check<=40) and (cobra_check1 >=500):
                        yoga_pose = 'cobra'
                        
                        if condition != 'cobra':
                            condition = 'cobra'
                            
                        
                        if (cobra_hip_angle >= cobra_hip_angle_avg+10):
                            state = "perfect"
                            
                            if count_condition == True:
                                count = count + 1
                                count_condition = False
                            
                    
                        elif  (cobra_hip_angle >= cobra_hip_angle_avg-5):
                            state = "standard"
                            
                        
                        else : 
                            state = "normal"
                            
                    else:
                        yoga_pose="nothing"    
                        count_condition = True
                        state = "normal"

            
            
                
                # 사람이 정면을 보고 있으면 all_body 상태에 tree 동작 감지
                else:
                    body_state = 'all_body'
                    if ((tree_left_angle <= 146 and tree_left_knee_angle <= 111) and (right_elbow[1]<right_shoulder[1] and left_elbow[1] < left_shoulder[1])) or ((tree_right_angle <= 141 and tree_right_knee_angle <= 106) and (left_elbow[1]<left_shoulder[1] and right_elbow[1] < right_shoulder[1])) and (tree_check <= 40):
                        yoga_pose="tree"
                        
                        if condition != 'tree':
                            condition = 'tree'
                            
                    
                        # 정확한 자세일 때는 'perfect' 상태로 저장
                        if (tree_right_angle <= 141-40 or tree_left_angle <= 146-40) and (tree_right_knee_angle <= 106-40 or tree_left_knee_angle <= 111-40):
                            state = "perfect"
                            
                            if count_condition == True:
                                count = count+1
                                count_condition = False
                            
                        # 일반적인 자세일 때는 'standard' 상태로 저장
                        elif (tree_right_angle <= 141-20 or tree_left_angle <= 146-20) and (tree_right_knee_angle <= 106-20 or tree_left_knee_angle <= 111-20):
                            state = "standard"
                        
                        # tree 자세라는 것이 감지되면 'normal' 상태로 저장
                        else : 
                            state = "normal"
        
                        
                    # 동작을 끝내면 다시 변수 초기화
                    elif (tree_right_angle >= 141 and tree_right_knee_angle >= 106) or (tree_left_angle >= 146 and tree_left_knee_angle >= 111):
                        yoga_pose="nothing"    
                        count_condition = True
                        state = "normal"
     

            except:
                pass

        
            # 상태창 표시하기
            cv2.rectangle(image, (0,0), (1200,150), (180,177,75), -1)

            
            if yoga_pose == "nothing":
               
                cv2.putText(image, 'Count', (15,20), font, 0.9, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, str(count), (20,80), font, 1.5, (0,0,0), 2, cv2.LINE_AA)
            
                # 몸의 각도 표시
                cv2.putText(image, 'pose', (120,20), font, 0.9, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, yoga_pose, (115,60), font, 0.9, (0,0,0), 2, cv2.LINE_AA)
        
                cv2.putText(image, 'state', (230,20), font, 0.9, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, state, (230,60), font, 0.9, (0,0,0), 2, cv2.LINE_AA)

                cv2.putText(image, 'right_angle : '+ str(tree_right_angle), (360,20), font, 0.7, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, 'right_knee_angle : '+ str(tree_right_knee_angle), (360,40), font, 0.7, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, 'right_wrist_position : '+ str(right_wrist[1]),  (360,60), font, 0.7, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, 'left_angle : '+ str(tree_left_angle), (360,80), font, 0.7, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, 'left_knee_angle : '+ str(tree_left_knee_angle),  (360,100), font, 0.7, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, 'left_wrist_position : '+ str(left_wrist[1]),  (360,120), font, 0.7, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, 'nose_position : '+ str(nose[1]),  (360,140), font, 0.7, (0,0,0), 1, cv2.LINE_AA)
 
 
            elif yoga_pose == 'tree':

                cv2.putText(image, 'Count', (15,20), font, 0.9, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, str(count), (20,80), font, 1.5, (0,0,0), 2, cv2.LINE_AA)
        
                # 몸의 각도 표시
                cv2.putText(image, 'pose', (120,20), font, 0.9, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, yoga_pose, (115,60), font, 0.9, (0,0,0), 2, cv2.LINE_AA)
        
                cv2.putText(image, 'state', (230,20), font, 0.9, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, state, (230,60), font, 0.9, (0,0,0), 2, cv2.LINE_AA)

                cv2.putText(image, 'right_angle : '+ str(tree_right_angle), (360,20), font, 0.7, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, 'right_knee_angle : '+ str(tree_right_knee_angle), (360,40), font, 0.7, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, 'right_wrist_position : '+ str(right_wrist[1]),  (360,60), font, 0.7, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, 'left_angle : '+ str(tree_left_angle), (360,80), font, 0.7, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, 'left_knee_angle : '+ str(tree_left_knee_angle),  (360,100), font, 0.7, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, 'left_wrist_position : '+ str(left_wrist[1]),  (360,120), font, 0.7, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, 'nose_position : '+ str(nose[1]),  (360,140), font, 0.7, (0,0,0), 1, cv2.LINE_AA)
                
                
                # 완벽한 자세일 경우
                if state == "perfect":
                    cv2.putText(image, str(int(tree_right_angle)), (right_hip[0] + 6, right_hip[1]), font, 0.9, light_green, 2)
                    cv2.putText(image, str(int(tree_right_knee_angle)), (right_knee[0] + 6, right_knee[1]), font, 0.9, light_green, 2)
                
                    cv2.putText(image, str(int(tree_left_angle)), (left_hip[0] + 6, left_hip[1]), font, 0.9, light_green, 2)
                    cv2.putText(image, str(int(tree_left_knee_angle)), (left_knee[0] + 6, left_knee[1]), font, 0.9, light_green, 2)
                    
                    
                    cv2.circle(image, (right_shoulder[0], right_shoulder[1]), 7, green, -1)
                    cv2.circle(image, (right_elbow[0], right_elbow[1]), 7, green, -1)
                    cv2.circle(image, (right_hip[0], right_hip[1]), 7, green, -1)
                    cv2.circle(image, (right_knee[0], right_knee[1]), 7, green, -1)
                    cv2.circle(image, (right_foot_index[0], right_foot_index[1]), 7, green, -1)
                    
                    
                    cv2.circle(image, (left_shoulder[0], left_shoulder[1]), 7, green, -1)
                    cv2.circle(image, (left_elbow[0], left_elbow[1]), 7, green, -1)
                    cv2.circle(image, (left_hip[0], left_hip[1]), 7, green, -1)
                    cv2.circle(image, (left_knee[0], left_knee[1]), 7, green, -1)
                    cv2.circle(image, (left_foot_index[0], left_foot_index[1]), 7, green, -1)


                    cv2.line(image, (right_elbow[0], right_elbow[1]), (right_shoulder[0], right_shoulder[1]), green, 4)
                    cv2.line(image, (right_shoulder[0], right_shoulder[1]), (right_hip[0], right_hip[1]), green, 4)
                    cv2.line(image, (right_hip[0], right_hip[1]), (right_knee[0], right_knee[1]), green, 4)
                    cv2.line(image, (right_knee[0], right_knee[1]), (right_heel[0], right_heel[1]), green, 4)
                    cv2.line(image, (right_heel[0], right_heel[1]), (right_foot_index[0], right_foot_index[1]), green, 4)
            
    
                    cv2.line(image, (left_elbow[0],left_elbow[1]), (left_shoulder[0],left_shoulder[1]), green, 4)
                    cv2.line(image, (left_shoulder[0],left_shoulder[1]), (left_hip[0], left_hip[1]), green, 4)
                    cv2.line(image, (left_hip[0], left_hip[1]), (left_knee[0], left_knee[1]), green, 4)
                    cv2.line(image, (left_knee[0], left_knee[1]), (left_heel[0], left_heel[1]), green, 4)
                    cv2.line(image, (left_heel[0], left_heel[1]), (left_foot_index[0], left_foot_index[1]), green, 4)
                
             
                # 일반적인 자세일 경우
                elif state == 'standard':
                    cv2.putText(image, str(int(tree_right_angle)), (right_hip[0] + 6, right_hip[1]), font, 0.9, yellow, 2)
                    cv2.putText(image, str(int(tree_right_knee_angle)), (right_knee[0] + 6, right_knee[1]), font, 0.9, yellow, 2)
            
                    cv2.putText(image, str(int(tree_left_angle)), (left_hip[0] + 6, left_hip[1]), font, 0.9,yellow, 2)
                    cv2.putText(image, str(int(tree_left_knee_angle)), (left_knee[0] + 6, left_knee[1]), font, 0.9, yellow, 2)

                
                    cv2.circle(image, (right_shoulder[0], right_shoulder[1]), 7, yellow, -1)
                    cv2.circle(image, (right_elbow[0], right_elbow[1]), 7, yellow, -1)
                    cv2.circle(image, (right_hip[0], right_hip[1]), 7,yellow, -1)
                    cv2.circle(image, (right_knee[0], right_knee[1]), 7, yellow, -1)
                    cv2.circle(image, (right_foot_index[0], right_foot_index[1]), 7, yellow, -1)
            
            
                    cv2.circle(image, (left_shoulder[0], left_shoulder[1]), 7, yellow, -1)
                    cv2.circle(image, (left_elbow[0], left_elbow[1]), 7, yellow, -1)
                    cv2.circle(image, (left_hip[0], left_hip[1]), 7, yellow, -1)
                    cv2.circle(image, (left_knee[0], left_knee[1]), 7, yellow, -1)
                    cv2.circle(image, (left_foot_index[0], left_foot_index[1]), 7, yellow, -1)

                    cv2.line(image, (right_elbow[0], right_elbow[1]), (right_shoulder[0], right_shoulder[1]), yellow, 4)
                    cv2.line(image, (right_shoulder[0], right_shoulder[1]), (right_hip[0], right_hip[1]), yellow, 4)
                    cv2.line(image, (right_hip[0], right_hip[1]), (right_knee[0], right_knee[1]), yellow, 4)
                    cv2.line(image, (right_knee[0], right_knee[1]), (right_heel[0], right_heel[1]), yellow, 4)
                    cv2.line(image, (right_heel[0], right_heel[1]), (right_foot_index[0], right_foot_index[1]), yellow, 4)

                    cv2.line(image, (left_elbow[0],left_elbow[1]), (left_shoulder[0],left_shoulder[1]), yellow, 4)
                    cv2.line(image, (left_shoulder[0],left_shoulder[1]), (left_hip[0], left_hip[1]), yellow, 4)
                    cv2.line(image, (left_hip[0], left_hip[1]), (left_knee[0], left_knee[1]), yellow, 4)
                    cv2.line(image, (left_knee[0], left_knee[1]), (left_heel[0], left_heel[1]), yellow, 4)
                    cv2.line(image, (left_heel[0], left_heel[1]), (left_foot_index[0], left_foot_index[1]), yellow, 4)
                
            
                # 처음 동작을 감지했을 떄
                elif state == 'normal':
                    cv2.putText(image, str(int(tree_right_angle)), (right_hip[0] + 6, right_hip[1]), font, 0.9, white, 2)
                    cv2.putText(image, str(int(tree_right_knee_angle)), (right_knee[0] + 6, right_knee[1]), font, 0.9, white, 2)
                
                    cv2.putText(image, str(int(tree_left_angle)), (left_hip[0] + 6, left_hip[1]), font, 0.9, white, 2)
                    cv2.putText(image, str(int(tree_left_knee_angle)), (left_knee[0] + 6, left_knee[1]), font, 0.9, white, 2)


                    cv2.circle(image, (right_shoulder[0], right_shoulder[1]), 7, white, -1)
                    cv2.circle(image, (right_elbow[0], right_elbow[1]), 7, white, -1)
                    cv2.circle(image, (right_hip[0], right_hip[1]), 7, white, -1)
                    cv2.circle(image, (right_knee[0], right_knee[1]), 7, white, -1)
                    cv2.circle(image, (right_foot_index[0], right_foot_index[1]), 7, white, -1)
            

                    cv2.circle(image, (left_shoulder[0], left_shoulder[1]), 7, white, -1)
                    cv2.circle(image, (left_elbow[0], left_elbow[1]), 7, white, -1)
                    cv2.circle(image, (left_hip[0], left_hip[1]), 7, white, -1)
                    cv2.circle(image, (left_knee[0], left_knee[1]), 7, white, -1)
                    cv2.circle(image, (left_foot_index[0], left_foot_index[1]), 7, white, -1)

                    cv2.line(image, (right_elbow[0], right_elbow[1]), (right_shoulder[0], right_shoulder[1]), white, 4)
                    cv2.line(image, (right_shoulder[0], right_shoulder[1]), (right_hip[0], right_hip[1]), white, 4)
                    cv2.line(image, (right_hip[0], right_hip[1]), (right_knee[0], right_knee[1]), white, 4)
                    cv2.line(image, (right_knee[0], right_knee[1]), (right_heel[0], right_heel[1]), white, 4)
                    cv2.line(image, (right_heel[0], right_heel[1]), (right_foot_index[0], right_foot_index[1]), white, 4)
            

                    cv2.line(image, (left_elbow[0],left_elbow[1]), (left_shoulder[0],left_shoulder[1]), white, 4)
                    cv2.line(image, (left_shoulder[0],left_shoulder[1]), (left_hip[0], left_hip[1]), white, 4)
                    cv2.line(image, (left_hip[0], left_hip[1]), (left_knee[0], left_knee[1]), white, 4)
                    cv2.line(image, (left_knee[0], left_knee[1]), (left_heel[0], left_heel[1]), white, 4)
                    cv2.line(image, (left_heel[0], left_heel[1]), (left_foot_index[0], left_foot_index[1]), white, 4)
             
                
            elif yoga_pose == 'cobra':

                cv2.putText(image, 'Count', (15,20), font, 0.9, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, str(count), (20,80), font, 1.5, (0,0,0), 2, cv2.LINE_AA)
        
                # 몸의 각도 표시
                cv2.putText(image, 'pose', (120,20), font, 0.9, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, yoga_pose, (115,60), font, 0.9, (0,0,0), 2, cv2.LINE_AA)
        
                cv2.putText(image, 'state', (230,20), font, 0.9, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, state, (230,60), font, 0.9, (0,0,0), 2, cv2.LINE_AA)

                cv2.putText(image, 'cobra_hip_angle : '+ str(cobra_hip_angle), (360,20), font, 0.7, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, 'cobra_hip_angle : '+ str(cobra_hip_angle), (360,40), font, 0.7, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, 'right_heel_position : '+ str(right_heel[1]),  (360,60), font, 0.7, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, 'left_heel_position : '+ str(left_heel[1]),  (360,80), font, 0.7, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, 'right_wrist_position : '+ str(right_wrist[1]),  (360,100), font, 0.7, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, 'left_wrist_position : '+ str(left_wrist[1]),  (360,120), font, 0.7, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, 'nose_position : '+ str(nose[1]),  (360,140), font, 0.7, (0,0,0), 1, cv2.LINE_AA)
                
                # 자세가 완벽할 경우
                if state == "perfect":
                    cv2.putText(image, str(int(cobra_hip_angle)), (right_hip[0] + 6, right_hip[1]), font, 0.9, light_green, 2)
                
                    cv2.circle(image, (nose[0], nose[1]), 7, green, -1)
                    cv2.circle(image, (right_wrist[0], right_wrist[1]), 7, green, -1)
                    cv2.circle(image, (right_shoulder[0], right_shoulder[1]), 7, green, -1)
                    cv2.circle(image, (right_elbow[0], right_elbow[1]), 7, green, -1)
                    cv2.circle(image, (right_hip[0], right_hip[1]), 7, green, -1)
                    cv2.circle(image, (right_knee[0], right_knee[1]), 7, green, -1)
                    cv2.circle(image, (right_foot_index[0], right_foot_index[1]), 7, green, -1)
            
                    cv2.line(image, (nose[0], nose[1]), (right_wrist[0], right_wrist[1]), green, 4)
                    cv2.line(image, (right_wrist[0], right_wrist[1]), (right_elbow[0], right_elbow[1]), green, 4)
                    cv2.line(image, (right_elbow[0], right_elbow[1]), (right_shoulder[0], right_shoulder[1]), green, 4)
                    cv2.line(image, (right_shoulder[0], right_shoulder[1]), (right_hip[0], right_hip[1]), green, 4)
                    cv2.line(image, (right_hip[0], right_hip[1]), (right_knee[0], right_knee[1]), green, 4)
                    cv2.line(image, (right_knee[0], right_knee[1]), (right_heel[0], right_heel[1]), green, 4)
                    cv2.line(image, (right_heel[0], right_heel[1]), (right_foot_index[0], right_foot_index[1]), green, 4)
        
                # 자세가 일반적인 수준일 경우
                elif state == 'standard':
                    cv2.putText(image, str(int(cobra_hip_angle)), (right_hip[0] + 6, right_hip[1]), font, 0.9, yellow, 2)
                
                    cv2.circle(image, (nose[0], nose[1]), 7, yellow, -1)
                    cv2.circle(image, (right_wrist[0], right_wrist[1]), 7, yellow, -1)
                    cv2.circle(image, (right_shoulder[0], right_shoulder[1]), 7, yellow, -1)
                    cv2.circle(image, (right_elbow[0], right_elbow[1]), 7, yellow, -1)
                    cv2.circle(image, (right_hip[0], right_hip[1]), 7, yellow, -1)
                    cv2.circle(image, (right_knee[0], right_knee[1]), 7, yellow, -1)
                    cv2.circle(image, (right_foot_index[0], right_foot_index[1]), 7, yellow, -1)
            
                    cv2.line(image, (nose[0], nose[1]), (right_wrist[0], right_wrist[1]), yellow, 4)
                    cv2.line(image, (right_wrist[0], right_wrist[1]), (right_elbow[0], right_elbow[1]), yellow, 4)
                    cv2.line(image, (right_elbow[0], right_elbow[1]), (right_shoulder[0], right_shoulder[1]), yellow, 4)
                    cv2.line(image, (right_shoulder[0], right_shoulder[1]), (right_hip[0], right_hip[1]), yellow, 4)
                    cv2.line(image, (right_hip[0], right_hip[1]), (right_knee[0], right_knee[1]), yellow, 4)
                    cv2.line(image, (right_knee[0], right_knee[1]), (right_heel[0], right_heel[1]), yellow, 4)
                    cv2.line(image, (right_heel[0], right_heel[1]), (right_foot_index[0], right_foot_index[1]), yellow, 4)
                
                
            
                # 코브라 자세임을 감지했을때
                elif state == 'normal':
                    cv2.putText(image, str(int(cobra_hip_angle)), (right_hip[0] + 6, right_hip[1]), font, 0.9, white, 2)
                   
                    cv2.circle(image, (nose[0], nose[1]), 7, white, -1)
                    cv2.circle(image, (right_wrist[0], right_wrist[1]), 7, white, -1)
                    cv2.circle(image, (right_shoulder[0], right_shoulder[1]), 7, white, -1)
                    cv2.circle(image, (right_elbow[0], right_elbow[1]), 7, white, -1)
                    cv2.circle(image, (right_hip[0], right_hip[1]), 7, white, -1)
                    cv2.circle(image, (right_knee[0], right_knee[1]), 7, white, -1)
                    cv2.circle(image, (right_foot_index[0], right_foot_index[1]), 7, white, -1)
            
                    cv2.line(image, (nose[0], nose[1]), (right_wrist[0], right_wrist[1]), white, 4)
                    cv2.line(image, (right_wrist[0], right_wrist[1]), (right_elbow[0], right_elbow[1]), white, 4)
                    cv2.line(image, (right_elbow[0], right_elbow[1]), (right_shoulder[0], right_shoulder[1]), white, 4)
                    cv2.line(image, (right_shoulder[0], right_shoulder[1]), (right_hip[0], right_hip[1]), white, 4)
                    cv2.line(image, (right_hip[0], right_hip[1]), (right_knee[0], right_knee[1]), white, 4)
                    cv2.line(image, (right_knee[0], right_knee[1]), (right_heel[0], right_heel[1]), white, 4)
                    cv2.line(image, (right_heel[0], right_heel[1]), (right_foot_index[0], right_foot_index[1]), white, 4)

                
            # tree동작 감지에서 측면을 보이고 있으면 측정불가 신호출력
            if body_state == 'side_body':
                if condition == 'tree':
                    
                        # 몸의 각도 표시
                        cv2.putText(image, 'pose', (120,20), font, 0.9, (0,0,0), 1, cv2.LINE_AA)
                        cv2.putText(image, yoga_pose, (115,60), font, 0.9, (0,0,0), 2, cv2.LINE_AA)
        
                        cv2.putText(image, 'state', (230,20), font, 0.9, (0,0,0), 1, cv2.LINE_AA)
                        cv2.putText(image, state, (230,60), font, 0.9, (0,0,0), 2, cv2.LINE_AA)

                        cv2.putText(image, 'right_angle : '+ str(tree_right_angle), (360,20), font, 0.7, (0,0,0), 1, cv2.LINE_AA)
                        cv2.putText(image, 'right_knee_angle : '+ str(tree_right_knee_angle), (360,40), font, 0.7, (0,0,0), 1, cv2.LINE_AA)
                        cv2.putText(image, 'right_wrist_position : '+ str(right_wrist[1]),  (360,60), font, 0.7, (0,0,0), 1, cv2.LINE_AA)
                        cv2.putText(image, 'left_angle : '+ str(tree_left_angle), (360,80), font, 0.7, (0,0,0), 1, cv2.LINE_AA)
                        cv2.putText(image, 'left_knee_angle : '+ str(tree_left_knee_angle),  (360,100), font, 0.7, (0,0,0), 1, cv2.LINE_AA)
                        cv2.putText(image, 'left_wrist_position : '+ str(left_wrist[1]),  (360,120), font, 0.7, (0,0,0), 1, cv2.LINE_AA)
                        cv2.putText(image, 'nose_position : '+ str(nose[1]),  (360,140), font, 0.7, (0,0,0), 1, cv2.LINE_AA)
                        cv2.putText(image, 'No!!', (590, 220), font, 2.0, red, 2)
                
           
                        cv2.circle(image, (right_shoulder[0], right_shoulder[1]), 7, red, -1)
                        cv2.circle(image, (right_elbow[0], right_elbow[1]), 7, red, -1)
                        cv2.circle(image, (right_hip[0], right_hip[1]), 7,red, -1)
                        cv2.circle(image, (right_knee[0], right_knee[1]), 7, red, -1)
                        cv2.circle(image, (right_foot_index[0], right_foot_index[1]), 7, red, -1)
            

                        cv2.circle(image, (left_shoulder[0], left_shoulder[1]), 7, red, -1)
                        cv2.circle(image, (left_elbow[0], left_elbow[1]), 7, red, -1)
                        cv2.circle(image, (left_hip[0], left_hip[1]), 7, red, -1)
                        cv2.circle(image, (left_knee[0], left_knee[1]), 7, red, -1)
                        cv2.circle(image, (left_foot_index[0], left_foot_index[1]), 7, red, -1)
            
                
                        cv2.line(image, (right_elbow[0], right_elbow[1]), (right_shoulder[0], right_shoulder[1]), red, 4)
                        cv2.line(image, (right_shoulder[0], right_shoulder[1]), (right_hip[0], right_hip[1]), red, 4)
                        cv2.line(image, (right_hip[0], right_hip[1]), (right_knee[0], right_knee[1]), red, 4)
                        cv2.line(image, (right_knee[0], right_knee[1]), (right_heel[0], right_heel[1]), red, 4)
                        cv2.line(image, (right_heel[0], right_heel[1]), (right_foot_index[0], right_foot_index[1]), red, 4)
            

                        cv2.line(image, (left_elbow[0],left_elbow[1]), (left_shoulder[0],left_shoulder[1]), red, 4)
                        cv2.line(image, (left_shoulder[0],left_shoulder[1]), (left_hip[0], left_hip[1]), red, 4)
                        cv2.line(image, (left_hip[0], left_hip[1]), (left_knee[0], left_knee[1]), red, 4)
                        cv2.line(image, (left_knee[0], left_knee[1]), (left_heel[0], left_heel[1]), red, 4)
                        cv2.line(image, (left_heel[0], left_heel[1]), (left_foot_index[0], left_foot_index[1]), red, 4)
            
                
            # pose detection 화면 출력
            cv2.imshow('Real-Time TREE Pose', image)
            if cv2.waitKey(10) & 0xFF == 27: #esc 키 종료
                break
        
        
        cap.release()
        cv2.destroyAllWindows()